In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog

from IPython.display import display

In [ ]:
def g_x(x, B, c, alpha):
    return (np.dot(x, B.T) @ B @ x)/2 + np.dot(c, x) + alpha


def df_dx(x, B, c):
    return np.dot(B.T, B) @ x + c


def find_xt(x, t, l, a, Δx):
    x_t = (x + t * (l + a * Δx))
    if (x_t < 0).any():
        t /= 2
        return find_xt(x, t, l, a, Δx)
    else:
        return x_t


def output(x_t, g_x, impr):
    x_t = np.array([format(x, '.2g') for x in x_t])
    xt = x_t.reshape((1, 8))
    cols = np.array([i for i in range (len(x_t))])
    xt_df = pd.DataFrame(data=xt, index=['x_t'], columns=cols)
    display(xt_df)

    df = pd.DataFrame(np.array([[ g_x, round(impr, 2)]]),
                   columns=['f(x_t)', 'Improvement, %'])
    display(df)


def convex(B0, c0, B, c, alpha, x):
    J0 = [j for j in range(len(x)) if x[j] == 0.0]
    g = np.array([g_x(x, B_i, c_i, alpha_i) for B_i, c_i, alpha_i in zip(B, c, alpha)])
   
    print('f(x) =', g_x(x, B0, c0, 0.0))

    I0 = np.array([i for i in range(len(g)) if g[i] == 0.0])
    df = df_dx(x, B0, c0)
    dg_I0 = np.array([df_dx(x, B[i], c[i]) for i in I0])

    d_m = np.zeros(len(x))
    d_m[np.delete(np.arange(len(x)), J0)] = -1

    x_1 = np.ones(len(x))
    l = linprog(df, A_ub = dg_I0, b_ub = np.zeros(len(dg_I0)), bounds=list(zip(d_m, x_1)))

    print ('dg/dx = ', l.fun)

    Δx = np.zeros(len(x)) - x
    k = np.dot(df, Δx)

    a0 = -l.fun/k
    a = a0/2 if (k < 0 < a0) else (a0 + 1 if (k > 0 and a0 > 0) else 1)

    x_t = find_xt(x, 2.0, l.x, a, Δx)
    
    res_gx = g_x(x_t, B0, c0, 0.0)
    impr = (100 - g_x(x_t, B0, c0, 0.0) / g_x(x, B0, c0, 0.0) * 100, 2)
    
    output(x_t, res_gx, impr[0])
    

In [ ]:
def main():
    B0 = np.array([[2, 1, 0, 4, 0, 3, 0, 0],
               [0, 4, 0, 3, 1, 1, 3, 2],
               [1, 3, 0, 5, 0, 4, 0, 4]])
  
    B = np.array([[[0, 0, 0.5, 2.5, 1, 0, -2.5, -2],
               [0.5, 0.5, -0.5, 0, 0.5, -0.5, -0.5, -0.5],
               [0.5, 0.5, 0.5, 0, 0.5, 1, 2.5, 4]],
              [[1.0, 2.0, -1.5, 3.0, -2.5, 0, -1, -0.5],
               [-1.5, -0.5, -1, 2.50, 3.5, 3, -1.5, -0.5],
               [1.5, 2.5, 1, 1.0, 2.5, 1.5, 3, 0]],
              [[0.75, 0.5, -1.0, 0.25, 0.25, 0, 0.25, 0.75],
               [-1.0, 1.0, 1.0, 0.75, 0.75, 0.5, 1, -0.75],
               [0.5, -0.25, 0.5, 0.75, 0.5, 1.250, -0.75, -0.25]],
              [[1.50, -1.50, -1.50, 2.0, 1.50, 0, 0.5, -1.50],
               [-0.5, -2.50, -0.5, -1.0, -2.50, 2.50, 1.0, 2.0],
               [-2.50, 1.0, -2.0, -1.50, -2.50, 0.5, 2.50, -2.50]],
              [[1.0, 0.25, -0.5, 1.250, 1.250, -0.5, 0.25, -0.75],
               [-1.0, -0.75, -0.75, 0.5, -0.25, 1.250, 0.25, -0.5],
               [0, 0.75, 0.5, -0.5, -1.0, 1.0, -1.0, 1.0]]])
  
    c0 = np.array([-1, -1, -1, -1, -2, 0, -2, -3])
  
    c = np.array([[0, 60, 80, 0, 0, 0, 40, 0],
              [2, 0, 3, 0, 2, 0, 3, 0],
              [0, 0, 80, 0, 0, 0, 0, 0],
              [0, -2, 1, 2, 0, 0, -2, 1],
              [-4, -2, 6, 0, 4, -2, 60, 2]])

    alpha = np.array([-51.75, -436.75, -33.7813, -303.375, -41.75])
    x = np.array([1, 0, 0, 2, 4, 2, 0, 0])
    convex(B0, c0, B, c, alpha, x)

main()

In [ ]:
def test():
    m = 3
    n = 8
    B0 = np.array([[2, 1, 0, 4, 0, 3, 0, 0],
            [0, 4, 0, 3, 1, 1, 3, 2],
            [1, 3, 0, 5, 0, 4, 0, 4]])
    B = np.array([[[0, 0, 0.5, 2.5, 1.0, 0, -2.5, -2.0],
            [0.5, 0.5, -0.5, 0, 0.5, -0.5, -0.5, -0.5],
            [0.5, 0.5, 0.5, 0, 0.5, 1.0, 2.5, 4.0]],
           [[1, 2, -1.5, 3, -2.5, 0, -1, -0.5],
            [-1.5, -0.5, -1, -2.5, 3.5, -3, -1.5, -0.5],
            [1.5, 2.5, -1, 1, 2.5, 1.5, 3, 0]],
           [[0.75, 0.5, -1, 0.25, 0.25, 0, 0.25, 0.75],
            [-1.0000, 1.0000, 4.0000, 0.7500, 0.7500, 0.5000, 7.0, -0.75],
            [0.5000, -0.2500, 0.5000, 0.75, 0.5000, 1.2500, -0.7500, -0.25]],
           [[1.5000, -1.5000, -1.5000, 2.0000, 1.5000, 0, 0.5000, -1.5000],
            [-0.5000, -2.5000, -0.5000, -5.0000, -2.5000, 3.5000, 1.0000, 2.0000],
            [-2.5000, 1.0000, -2.0000, -1.500, -2.5000, 0.5000, 8.5000, -2.5000]],
           [[1.0000, 0.2500, -0.5000, 1.2500, 1.2500, -0.5000, 0.2500, -0.7500],
            [-1.0000, -0.7500, -0.7500, 0.5000, -0.2500, 1.2500, 0.2500, -0.5000],
            [0, 0.7500, 0.5000, -0.5000, -1.0000, 1.0000, -1.0000, 1.0000]]])
    c0 = np.array([-1, -1, -1, -1, -2, 0, -2, -3])
    c = np.array([[0, 60, 80, 0, 0, 0, 40, 0],
           [2, 0, 3, 0, 2, 0, 3, 0],
           [0, 0, 80, 0, 0, 0, 0, 0],
           [0, -2, 1, 2, 0, 0, -2, 1],
           [-4, -2, 6, 0, 4, -2, 60, 2]])
    alpha = np.array([-687.125, -666.625, -349.5938, -254.625, -45.1563])
    x = np.array([0, 8, 2, 1, 0, 4, 0, 0])

    convex(B0, c0, B, c, alpha, x)